In [3]:
import os
import torch
import argparse
import numpy as np
import torch.utils.data

from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image


class AutoEncoder(nn.Module):
    def __init__(self, inp_size, hid_size):
        super(AutoEncoder, self).__init__()
        """
        Here you should define layers of your autoencoder
        Please note, if a layer has trainable parameters, it should be nn.Linear. 
        ## !! CONVOLUTIONAL LAYERS CAN NOT BE HERE !! ##
        However, you can use any noise inducing layers, e.g. Dropout.

        Your network must not have more than six layers with trainable parameters.
        :param inp_size: integer, dimension of the input object
        :param hid_size: integer, dimension of the hidden representation
        """
   
        self.fc1 = nn.Linear(inp_size, ((inp_size + hid_size)*2)//3)
        self.fc2 = nn.Linear(((inp_size + hid_size)*2)//3, (inp_size + hid_size)//3)
        self.fc3 = nn.Linear((inp_size + hid_size)//3, hid_size)
        
        self.fc4 = nn.Linear(hid_size, (inp_size + hid_size)//3)
        self.fc5 = nn.Linear((inp_size + hid_size)//3, ((inp_size + hid_size)*2)//3)
        self.fc6 = nn.Linear(((inp_size + hid_size)*2)//3, inp_size)
        
        self.drop = nn.Dropout()

    def encode(self, x):
        """
        Encodes objects to hidden representations (E: R^inp_size -> R^hid_size)

        :param x: inputs, Variable of shape (batch_size, inp_size)
        :return:  hidden represenation of the objects, Variable of shape (batch_size, hid_size)
        """
        
        x = self.drop(nn.functional.relu(self.fc1(x)))
        x = self.drop(nn.functional.relu(self.fc2(x)))
        x = nn.functional.relu(self.fc3(x))
        return x

    def decode(self, h):
        """
        Decodes objects from hidden representations (D: R^hid_size -> R^inp_size)

        :param h: hidden represenatations, Variable of shape (batch_size, hid_size)
        :return:  reconstructed objects, Variable of shape (batch_size, inp_size)
        """
        x = self.drop(nn.functional.relu(self.fc4(h)))
        x = self.drop(nn.functional.relu(self.fc5(x)))
        x = nn.functional.relu(self.fc6(x))
        return x

    def forward(self, x):
        """
        Encodes inputs to hidden representations and decodes back.

        x: inputs, Variable of shape (batch_size, inp_size)
        return: reconstructed objects, Variable of shape (batch_size, inp_size)
        """
        return self.decode(self.encode(x))

    def loss_function(self, recon_x, x, lam = 1e-4):
        """
        Calculates the loss function.

        :params recon_x: reconstructed object, Variable of shape (batch_size, inp_size)
        :params x: original object, Variable of shape (batch_size, inp_size)
        :return: loss
        """
        l1_crit = nn.L1Loss(size_average=False)
        l1_reg = 0
        for param in self.parameters():
            l1_reg += l1_crit(param, target=torch.zeros_like(param))

        return torch.sum(torch.pow(recon_x - x,2)) / recon_x.shape[0] + lam * l1_reg

In [4]:
def train(model, optimizer, train_loader, test_loader, n_ep = 10):
    for epoch in range(n_ep):
        model.train()
        train_loss, test_loss = 0, 0
        for data, _ in train_loader:
            data = Variable(data).view(-1, 784)
            x_rec = model(data)
            loss = model.loss_function(x_rec, data)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.data[0]
        print('=> Epoch: %s Average loss: %.3f' % (epoch, train_loss / len(train_loader.dataset)))

        model.eval()
        for data, _ in test_loader:
            data = Variable(data, volatile=True).view(-1, 784)
            x_rec = model(data)
            test_loss += model.loss_function(x_rec, data).data[0]

        test_loss /= len(test_loader.dataset)
        print('=> Test set loss: %.3f' % test_loss)

        n = min(data.size(0), 8)
        comparison = torch.cat([data.view(-1, 1, 28, 28)[:n], x_rec.view(-1, 1, 28, 28)[:n]])
        if not os.path.exists('./pics'): os.makedirs('./pics')
        save_image(comparison.data.cpu(), 'pics/reconstruction_' + str(epoch) + '.png', nrow=n)
    return model

In [5]:
def test_work():
    print('Start test')
    get_loader = lambda train: torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=train, download=True, transform=transforms.ToTensor()),
        batch_size=50, shuffle=True)
    train_loader, test_loader = get_loader(True), get_loader(False)
    
    try:
        model = AutoEncoder(inp_size=784, hid_size=20)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
    except Exception:
        assert False, 'Error during model creation'
        return

    try:
        model = train(model, optimizer, train_loader, test_loader)
    except Exception:
        assert False, 'Error during training'
        return

    test_x = Variable(torch.randn(1, 784))    
    rec_x, hid_x = model(test_x), model.encode(test_x)
    submodules = dict(model.named_children())
    layers_with_params = np.unique(['.'.join(n.split('.')[:-1]) for n, _ in model.named_parameters()])
    
    assert (hid_x.dim() == 2) and (hid_x.size(1) == 20),  'Hidden representation size must be equal to 20'
    assert (rec_x.dim() == 2) and (rec_x.size(1) == 784), 'Reconstruction size must be equal to 784'
    assert len(layers_with_params) <= 6, 'The model must have no more than 6 layers '
    assert np.all(np.concatenate([list(p.shape) for p in model.parameters()]) <= 800), 'All hidden sizes must be less than 800'
    assert np.all([isinstance(submodules[name], nn.Linear) for name in layers_with_params]), 'All layers with parameters must be nn.Linear'
    print('Success!🎉')


In [6]:
test_work()

Start test
=> Epoch: 0 Average loss: 0.737
=> Test set loss: 0.541
=> Epoch: 1 Average loss: 0.615
=> Test set loss: 0.493
=> Epoch: 2 Average loss: 0.590
=> Test set loss: 0.471
=> Epoch: 3 Average loss: 0.581
=> Test set loss: 0.456
=> Epoch: 4 Average loss: 0.574
=> Test set loss: 0.451
=> Epoch: 5 Average loss: 0.569
=> Test set loss: 0.445
=> Epoch: 6 Average loss: 0.565
=> Test set loss: 0.436
=> Epoch: 7 Average loss: 0.562
=> Test set loss: 0.433
=> Epoch: 8 Average loss: 0.559
=> Test set loss: 0.436
=> Epoch: 9 Average loss: 0.557
=> Test set loss: 0.429
Success!🎉


In [10]:
model = AutoEncoder(inp_size=784, hid_size=20)
model.encode

<bound method AutoEncoder.encode of AutoEncoder(
  (fc1): Linear(in_features=784, out_features=536, bias=True)
  (fc2): Linear(in_features=536, out_features=268, bias=True)
  (fc3): Linear(in_features=268, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=268, bias=True)
  (fc5): Linear(in_features=268, out_features=536, bias=True)
  (fc6): Linear(in_features=536, out_features=784, bias=True)
  (drop): Dropout(p=0.5)
)>